In [14]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [2]:
doctors = pd.read_csv('data/DocGraph_Hop_Teaming_2018.csv', nrows = 100)
doctors

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508062167,1730166109,350,370,53.922,72.612
1,1508065640,1730166109,25,25,49.800,55.006
2,1508052093,1730166109,16,16,109.500,70.593
3,1508172545,1730166109,14,14,103.357,75.483
4,1508285131,1730166109,20,21,89.952,89.880
...,...,...,...,...,...,...
95,1508178229,1730166893,31,32,67.125,61.279
96,1508196445,1730166893,23,24,60.833,59.129
97,1508811076,1730166935,14,15,62.533,62.827
98,1508871252,1730166935,29,31,25.323,36.693


In [17]:
db = sqlite3.connect('data/doc_grahp_hop.sqlite')

chunks = pd.read_csv('data/DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)
hop = []
for chunk in chunks: 
    chunk = chunk[(chunk['transaction_count'] >= 50) & (chunk['average_day_wait'] < 50)]
    hop.append(chunk)

In [29]:
chunks.to_sql('hop', db, if_exists = 'append', index = False)

AttributeError: 'TextFileReader' object has no attribute 'to_sql'

In [22]:
chunk

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
217910000,1417376278,1497901458,45,69,34.043,53.749
217910002,1417282138,1497901458,53,73,36.288,52.800
217910004,1417266537,1497901458,109,183,42.656,43.998
217910008,1417239781,1497901458,152,155,34.568,60.137
217910012,1417177403,1497901458,48,66,44.182,62.360
...,...,...,...,...,...,...
217911281,1417037664,1497939599,36,106,19.330,42.407
217911283,1417194903,1497939599,22,70,16.629,30.598
217911284,1417406372,1497939599,21,65,20.123,37.750
217911298,1417064825,1497940605,75,79,10.418,34.744
